In [8]:
import pathlib
import networkx as nx
from typing import Callable, Dict, Hashable, Set, Tuple, Union

from helpers import get_network, WorkflowTaskGraph
from heft_synthesis import heft
from wfcommons.wfchef.recipes import MontageRecipe

workflow = MontageRecipe(num_tasks=100).build_workflow()
task_graph = WorkflowTaskGraph(workflow)
network = get_network()

In [14]:
# Custom comparison function for modified heft
task_order = list(nx.topological_sort(task_graph.to_networkx()))
node_order = network.nodes

comp = task_graph.computation_matrix(network).loc[task_order,node_order]
data = task_graph.data_matrix().loc[task_order,task_order]
comm = network.communication_matrix().loc[node_order,node_order]

def modified_heft_comparison(selected_nodes: Set[int],
                             node: Tuple[int, float], 
                             other: Tuple[int, float]) -> Union[float, int]:
        node_idx, node_eft = node
        other_idx, other_eft = other
        eft_diff = node_eft - other_eft

        selected_names = [node_order[idx] for idx in selected_nodes]
        node_cost = 0.0
        for n in [node_order[node_idx], *selected_names]:
            if n == "__satellite__":
                continue
            node_cost += network.node_cost[network._graph.nodes[n]["speed"]]

        other_cost = 0.0
        for n in [node_order[other_idx], *selected_names]:
            if n == "__satellite__":
                continue
            other_cost += network.node_cost[network._graph.nodes[n]["speed"]]

        cost_diff = node_cost - other_cost
        return eft_diff + cost_diff

schedule = heft(
    data=data.values, 
    comp=comp.values.T, 
    comm=comm.values, 
    compare=modified_heft_comparison
)

modified_heft_subnetwork = network.subnetwork(
    [node_order[proc] for proc, tasks in schedule.items() if len(tasks) >= 1]
)

print(f'Modified HEFT subnetwork cost: {modified_heft_subnetwork.cost()}')


{0, 1, 2, 3, 4}


TypeError: 'int' object is not subscriptable